In [1]:
import numpy as np
import matplotlib
import xarray as xr

In [15]:
from AFL.double_agent.AutoSAS import *
from AFL.double_agent.AutoSAS_Driver import *
from AFL.double_agent.Labeler import *
from AFL.double_agent.Pipeline import Pipeline
from AFL.double_agent.Preprocessor import *

In [18]:
ds_in = xr.load_dataset('./AllAutoSAS.netcdf')

# Build the AutoSAS models with the following format
models
|_model_1
    |_ name: str (required)
    |_ sasmodel: str (required)
    |_ q_min: float (optional)
    |_ q_max: float (optional)
    |_ fit_parameters: required
        |_ param_1:
            |_ value: type
            |_ bounds: (min, max) or None
        |_ ...
        |_ param_n:
        

In [16]:
initial_models = [
    {'name':"PGC",
     'sasmodel':'poly_gauss_coil',
     'fit_params':{
            "scale"       : {'value':0.01,  'bounds':(1e-6,1e4)},
            "i_zero"      : {'value':1,   'bounds':None},
            "background"  : {'value':1e-6,       'bounds':(1e-6,1e4)},
            "rg"          : {'value':40,       'bounds':(1e-1,1e2)},
            "polydispersity": {'value':1.,       'bounds':None}
        }
    },
    {'name':"mixed_model",
     'sasmodel':'sphere+poly_gauss_coil',
     'fit_params':{
            "scale"       : {'value':1,   'bounds':None},
            "background"  : {'value':1e-6,       'bounds':(1e-6,1e2)},
            "A_scale"       : {'value':1e-3,   'bounds':(1e-6,1e1)},
            "A_radius"      : {'value':50,       'bounds':(1e-1,1e2)},
            "A_radius_pd"   : {'value':0.15,       'bounds':(0.1,0.5)},
            "A_sld"         : {'value':9.857,     'bounds':None},
            "A_sld_solvent" : {'value':9.455,     'bounds':None},
         
            "B_scale"       : {'value':1e-3,  'bounds':(1e-6,1e1)},
            "B_i_zero"      : {'value':1,   'bounds':None},
            "B_rg"          : {'value':40,       'bounds':(1e-1,1e2)},
        }
    },
    {'name':"sphere",
     'sasmodel':'sphere',
     'fit_params':{
            "scale"       : {'value':1e-3,   'bounds':(1e-4,1e2)},
            "background"  : {'value':1e-6,       'bounds':(1e-6,1e2)},
            "radius"      : {'value':50,       'bounds':(1e-1,1e2)},
            "radius_pd"   : {'value':0.5,     'bounds':None},
            "radius_pd"   : {'value':0.15,       'bounds':(0.1,0.5)},
            "sld"         : {'value':9.857,     'bounds':None},
            "sld_solvent" : {'value':9.455,     'bounds':None},
        }
    },
]



## remote operation

- start the server using python ~/AFL-agent.double_agent.AutoSAS_Driver.py on the remote location
- connect this notebook to the server using the AFL-client base with the following code

In [10]:
# sasfit_url = 'localhost:5058'
# sasfit_client = Client(sasfit_url.split(':')[0],port=sasfit_url.split(':')[1])
# sasfit_client.login('SampleServer_SASfit_Client')
# sasfit_client.debug(False)

# sasfit_client.set_config(
#     filepath='/path/to/filepath/or/save/',
#     save_path='/path/to/filepath/or/save/',
#     model_inputs = initial_models
# )

## Local operation does not require the server 

In [25]:
with Pipeline() as pline1:
    AutoSAS(
        sas_variable = 'I',
        sas_err_variable = 'dI',
        q_variable = 'q',
        output_prefix = 'SAXS',
        sas_resolution_variable = None,
        sample_dim = 'sample',
        model_dim = 'models',
        model_inputs = initial_models,
        fit_method = None, #default is LM
        server_id = None,
        q_min = None,
        q_max = None,
        name = 'AutoSAS',
    )

    VarsToArray(
        input_variables=['BzOH','PhOH','P188'],
        output_variable='soln_comps',
        variable_dim='comps'
    )
    
    ModelSelectBestChiSq(
        all_chisq_var='SAXS_all_chisq',
        model_names_var='models',
        sample_dim='sample',
        output_prefix='BestChiSq',
        name='BestChiSq'
    )

    ModelSelectParsimony(
        all_chisq_var='SAXS_all_chisq',
        model_names_var='models',
        sample_dim='sample',
        cutoff_threshold=1.,
        model_complexity=[0,2,1],
        model_inputs= initial_models,
        output_prefix='Parsimony',
        name='ModelSelection_Parsimony'
    )

    ModelSelectAIC(
        all_chisq_var='SAXS_all_chisq',
        model_names_var='models',
        sample_dim='sample',
        model_inputs = initial_models,
        output_prefix='AutoSAS_AIC',
        name='AIC'
    )


# pline1.draw
    

['BzOH', 'PhOH', 'P188'] soln_comps


In [26]:
dso = pline1.calculate(ds_in)

  0%|          | 0/5 [00:00<?, ?it/s]

soln_comps
<xarray.DataArray (comps: 3, sample: 179)> Size: 4kB
array([[ 0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,
         0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  9. ,  9. ,  9. ,  9. ,
         9. ,  9. ,  9. ,  9. ,  9. , 18. , 18. , 18. , 18. ,  0. ,  0. ,
         0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  9. ,  9. ,  9. ,  9. ,  9. ,
         9. ,  9. ,  9. ,  9. ,  9. ,  9. ,  9. ,  9. ,  9. ,  9. ,  9. ,
         9. ,  5. ,  5. ,  5. ,  5. ,  5. ,  5. ,  5. ,  5. ,  5. ,  5. ,
         5. ,  5. ,  5. ,  5. ,  5. ,  5. ,  5. ,  5. ,  5. ,  5. ,  5. ,
         5. ,  5. ,  5. ,  5. ,  5. ,  5. ,  5. ,  5. ,  5. ,  5. ,  5. ,
         5. ,  5. ,  5. ,  5. ,  5. ,  5. ,  5. ,  5. , 14. , 14. , 14. ,
        14. , 14. , 14. , 14. , 14. , 14. , 14. , 14. , 14. , 14. , 14. ,
        14. , 14. , 14. , 14. , 14. , 14. , 14. , 14. , 14. , 14. , 14. ,
        14. , 14. , 14. , 14. , 14. , 18. , 18. , 18. , 18. , 18. , 18. ,
        18. , 18. , 18. , 18. , 18. , 18. , 18. 

# visualization

In [28]:
from AFL.automation.shared.DatasetWidget import DatasetWidget

widget = DatasetWidget(
    dataset=dso,
    sample_dim='sample',
    # scatt_variables=['sans','fit_I_mixed_model'],
    # comps_variable='sans_comps',
    # comps_color_variable='AIC_labels',
    xmin=0.007,
    xmax=0.5
)
widget.run()